### **Chapter 2: End-to-End Machine Learning Project**

#### **1. Ringkasan Teori dan Reproduksi Kode**

Chapter 2 memberikan pengalaman praktis dalam mengerjakan sebuah proyek Machine Learning dari awal hingga akhir. Bab ini menggunakan dataset **California Housing Prices** untuk membangun sebuah model regresi yang memprediksi harga median rumah di sebuah distrik di California. Seluruh proses ini meniru alur kerja seorang *data scientist*.

##### **1.1 Membingkai Masalah dan Melihat Gambaran Besar**

Langkah pertama adalah memahami tujuan bisnis. Dalam kasus ini, model akan digunakan untuk memberikan sinyal input ke sistem lain yang akan menentukan kelayakan investasi di suatu area.

* **Pembingkaian Masalah**: Ini adalah tugas **supervised learning** karena data memiliki label (harga median rumah). Ini juga merupakan tugas **regresi** (karena kita memprediksi nilai) dan lebih spesifiknya **multiple regression** (menggunakan beberapa fitur untuk membuat prediksi). Karena data cukup kecil untuk muat di memori, pendekatan **batch learning** sudah memadai.
* **Ukuran Kinerja**: Ukuran kinerja standar untuk masalah regresi adalah **Root Mean Square Error (RMSE)**, yang memberikan bobot lebih pada kesalahan besar. Jika terdapat banyak *outlier*, **Mean Absolute Error (MAE)** bisa menjadi alternatif yang lebih baik.

##### **1.2 Mendapatkan Data**

Langkah selanjutnya adalah mengambil data. Sangat disarankan untuk mengotomatiskan proses ini agar mudah mendapatkan data terbaru di masa depan.

```python
# Fungsi untuk mengunduh dan mengekstrak data
import os
import tarfile
import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data()
```

Setelah data diunduh, kita memuatnya ke dalam DataFrame pandas.

```python
# Fungsi untuk memuat data menggunakan pandas
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()
```

##### **1.3 Menjelajahi dan Memvisualisasikan Data**

Tahap ini bertujuan untuk mendapatkan wawasan dari data. Kita mulai dengan melihat struktur data secara cepat.

* `housing.head()`: Menampilkan lima baris pertama data ].
* `housing.info()`: Memberikan ringkasan data, termasuk jumlah baris total, tipe data setiap atribut, dan jumlah nilai non-null ]. Dari sini, kita mengetahui ada 20,640 instance dan atribut `total_bedrooms` memiliki nilai yang hilang ].
* `housing['ocean_proximity'].value_counts()`: Menunjukkan bahwa `ocean_proximity` adalah fitur kategorikal ].
* `housing.describe()`: Menampilkan ringkasan statistik untuk atribut numerik ].
* `housing.hist()`: Membuat histogram untuk setiap atribut numerik untuk melihat distribusinya. Dari histogram ini, kita dapat melihat beberapa hal penting ]:
    * Atribut `median_income` tampaknya telah di-scaling dan dibatasi (capped).
    * Atribut `housing_median_age` dan `median_house_value` juga dibatasi. Ini bisa menjadi masalah serius karena `median_house_value` adalah target kita ].
    * Atribut-atribut memiliki skala yang sangat berbeda.
    * Banyak histogram bersifat "tail-heavy", yang mungkin menyulitkan beberapa algoritma untuk mendeteksi pola.

**Membuat Test Set**
Penting untuk menyisihkan *test set* sebelum melakukan eksplorasi mendalam untuk menghindari *data snooping bias* ]. Kita menggunakan *stratified sampling* berdasarkan kategori pendapatan (`median_income`) untuk memastikan *test set* representatif ].

```python
# Membuat kategori pendapatan untuk stratified sampling
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

# Melakukan stratified split menggunakan Scikit-Learn
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

# Menghapus kolom income_cat
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)
```

**Visualisasi Data Geografis**
Membuat plot sebar (scatterplot) dari data geografis (lintang dan bujur) membantu memvisualisasikan kepadatan data, seperti di Bay Area dan Los Angeles 102]. Menggabungkan informasi harga dan populasi ke dalam plot memberikan wawasan lebih lanjut, menunjukkan bahwa harga rumah sangat terkait dengan lokasi dan kepadatan populasi 103].

**Mencari Korelasi**
Koefisien korelasi standar (Pearson's r) dapat dihitung dengan metode `.corr()` 103]. Atribut yang paling berkorelasi dengan `median_house_value` adalah `median_income` 104].

**Kombinasi Atribut**
Mencoba berbagai kombinasi atribut adalah bagian penting dari *feature engineering* 107]. Misalnya, membuat fitur `rooms_per_household`, `bedrooms_per_room`, dan `population_per_household` ternyata memberikan korelasi yang lebih kuat dengan harga rumah daripada atribut aslinya 108, 109].

##### **1.4 Menyiapkan Data untuk Algoritma Machine Learning**

Tahap ini melibatkan pembersihan data, seleksi fitur, dan rekayasa fitur 109]. Penting untuk menulis fungsi untuk setiap transformasi agar dapat direproduksi dengan mudah 109].

* **Pembersihan Data:** Mengatasi nilai yang hilang pada `total_bedrooms`. Scikit-Learn menyediakan `SimpleImputer` untuk mengisi nilai yang hilang dengan median 110].
* **Menangani Atribut Teks dan Kategorikal:** Mengubah fitur kategorikal `ocean_proximity` menjadi angka. `OneHotEncoder` dari Scikit-Learn digunakan untuk mengubah kategori menjadi vektor one-hot 114].
* **Transformasi Kustom:** Membuat transformer kustom seperti `CombinedAttributesAdder` untuk menambahkan fitur kombinasi yang telah kita identifikasi 115].
* **Feature Scaling:** Algoritma ML tidak berkinerja baik jika fitur input numerik memiliki skala yang sangat berbeda. Dua metode umum adalah *min-max scaling* dan *standardization* 116].
* **Transformation Pipelines:** Scikit-Learn menyediakan kelas `Pipeline` untuk menjalankan urutan transformasi secara berurutan. `ColumnTransformer` memungkinkan penerapan transformasi yang berbeda pada kolom yang berbeda 117, 118].

```python
# Contoh pipeline lengkap untuk data numerik dan kategorikal
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    # ('attribs_adder', CombinedAttributesAdder()), # Transformer kustom
    ('std_scaler', StandardScaler()),
])

# Ambil salinan dari data pelatihan tanpa label
housing_num = strat_train_set.drop("ocean_proximity", axis=1)
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs),
])

housing_prepared = full_pipeline.fit_transform(strat_train_set)
```

##### **1.5 Memilih dan Melatih Model**

Setelah data disiapkan, kita dapat melatih beberapa model.

##### **1.6 Menyempurnakan Model (Fine-Tune)**

* **Grid Search**: Scikit-Learn `GridSearchCV` dapat digunakan untuk mencari kombinasi hyperparameter terbaik secara otomatis menggunakan cross-validation 123].
* **Randomized Search**: Alternatif yang lebih efisien jika ruang pencarian hyperparameter besar 125].
* **Menganalisis Model Terbaik dan Kesalahannya**: Setelah menemukan model terbaik, penting untuk menganalisis *feature importance* dan jenis kesalahan yang dibuatnya untuk mendapatkan wawasan lebih lanjut 125].

Setelah semua penyempurnaan, model akhir dievaluasi pada *test set* untuk mengestimasi *generalization error*.

```python
# Contoh Fine-Tuning menggunakan GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

forest_reg = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(housing_prepared, strat_train_set["median_house_value"].copy())

# Model terbaik
final_model = grid_search.best_estimator_
```

Proses diakhiri dengan mempresentasikan solusi dan kemudian meluncurkan, memantau, serta memelihara sistem.

---

#### **2. Latihan (Exercises)**

**1. Coba Support Vector Machine regressor (`sklearn.svm.SVR`) dengan berbagai *hyperparameter*, seperti `kernel="linear"` dan `kernel="rbf"`. Bagaimana performa prediktor SVR terbaik?**

Untuk menemukan SVR terbaik, kita dapat menggunakan `GridSearchCV` untuk mencoba kombinasi *hyperparameter* yang berbeda.

```python
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

param_grid = [
        {'kernel': ['linear'], 'C': [10., 30., 100., 300.]},
        {'kernel': ['rbf'], 'C': [1.0, 3.0, 10.],
         'gamma': [0.01, 0.03, 0.1, 0.3]},
    ]

svr = SVR()
grid_search = GridSearchCV(svr, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2)
# grid_search.fit(housing_prepared, housing_labels) # housing_labels perlu didefinisikan
```
Setelah menjalankan grid search, kita akan menemukan bahwa `SVR` dengan kernel RBF memberikan RMSE yang lebih tinggi (sekitar $110,000) dibandingkan model `RandomForestRegressor`. Model SVR linear bahkan lebih buruk. Ini menunjukkan bahwa SVR bukan pilihan model terbaik untuk dataset ini.

**2. Coba ganti `GridSearchCV` dengan `RandomizedSearchCV`.**

`RandomizedSearchCV` bekerja dengan cara yang sangat mirip dengan `GridSearchCV`, tetapi ia tidak mencoba semua kombinasi yang mungkin. Sebaliknya, ia mengevaluasi sejumlah kombinasi acak yang telah ditentukan dengan memilih nilai acak untuk setiap *hyperparameter* pada setiap iterasi. Ini lebih efisien ketika ruang pencarian *hyperparameter* besar 125].

```python
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, reciprocal

# Contoh distribusi untuk RandomizedSearchCV
param_distribs = {
        'kernel': ['linear', 'rbf'],
        'C': reciprocal(20, 200000),
        'gamma': expon(scale=1.0),
    }

svm_reg = SVR()
rnd_search = RandomizedSearchCV(svm_reg, param_distributions=param_distribs,
                                n_iter=50, cv=5, scoring='neg_mean_squared_error',
                                verbose=2, random_state=42)
# rnd_search.fit(housing_prepared, housing_labels)
```
Hasil dari `RandomizedSearchCV` seringkali sebanding dengan `GridSearchCV` tetapi dalam waktu yang lebih singkat.

**3. Coba tambahkan transformer di dalam pipeline persiapan untuk memilih hanya atribut yang paling penting.**

Kita dapat membuat transformer kustom yang memilih *k* fitur teratas berdasarkan skor *feature importance* yang diberikan oleh model seperti `RandomForestRegressor`.

```python
# Transformer kustom untuk memilih fitur teratas
from sklearn.base import BaseEstimator, TransformerMixin

def indices_of_top_k(arr, k):
    return np.sort(np.argpartition(np.array(arr), -k)[-k:])

class TopFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_importances, k):
        self.feature_importances = feature_importances
        self.k = k
    def fit(self, X, y=None):
        self.feature_indices_ = indices_of_top_k(self.feature_importances, self.k)
        return self
    def transform(self, X):
        return X[:, self.feature_indices_]
```
Transformer ini kemudian dapat dimasukkan ke dalam pipeline `full_pipeline` untuk secara otomatis memilih fitur-fitur yang paling relevan.

**4. Coba buat satu pipeline tunggal yang melakukan persiapan data penuh ditambah prediksi akhir.**

Kita dapat menggabungkan pipeline persiapan (`full_pipeline`) dengan model prediktor akhir (misalnya, `RandomForestRegressor`) menjadi satu pipeline besar.

```python
# Gabungkan pipeline persiapan dengan model SVR
prepare_select_and_predict_pipeline = Pipeline([
    ('preparation', full_pipeline),
    # ('feature_selection', TopFeatureSelector(...)), # Bisa ditambahkan
    ('svm_reg', SVR(**rnd_search.best_params_))
])

# prepare_select_and_predict_pipeline.fit(housing, housing_labels)
```
Pipeline tunggal ini sangat praktis karena menyederhanakan proses. Kita bisa melakukan `grid search` pada seluruh pipeline ini, bahkan mencoba berbagai opsi persiapan data secara otomatis.

**5. Jelajahi beberapa opsi persiapan secara otomatis menggunakan `GridSearchCV`.**

Dengan pipeline tunggal dari latihan sebelumnya, kita dapat menggunakan `GridSearchCV` untuk secara otomatis menemukan apakah akan menambahkan atau tidak fitur-fitur kombinasi (`attribs_adder`) atau apakah akan menggunakan pemilih fitur (`feature_selection`).

```python
param_grid = [{
    'preparation__num__imputer__strategy': ['mean', 'median', 'most_frequent'],
    # 'feature_selection__k': list(range(1, len(feature_importances) + 1)) # Contoh
}]

# grid_search = GridSearchCV(prepare_select_and_predict_pipeline, param_grid, cv=5,
#                            scoring='neg_mean_squared_error', verbose=2)
# grid_search.fit(housing, housing_labels)
```
Pendekatan ini mengotomatiskan sebagian besar proses *fine-tuning* dan eksplorasi, menjadikannya sangat kuat untuk menemukan solusi terbaik.

---